In [1]:
import pandas as pd

In [2]:
# плохо считывалось из-за формата, пришлось все в строку сделать
rfm_analysis = pd.read_csv('https://stepik.org/media/attachments/lesson/413464/RFM_ht_data.csv', dtype=str)

In [3]:
# меняем цифровой столбец на float
rfm_analysis = rfm_analysis.astype({'Amount': float})

In [4]:
# дату тоже форматируем
rfm_analysis.InvoiceDate = pd.to_datetime(rfm_analysis.InvoiceDate)

In [5]:
# проверяем
rfm_analysis.dtypes

InvoiceNo               object
CustomerCode            object
InvoiceDate     datetime64[ns]
Amount                 float64
dtype: object

In [6]:
rfm_analysis.head(2)

,InvoiceNo,CustomerCode,InvoiceDate,Amount
0,C0011810010001,19067290,2020-09-01,1716.00
1,C0011810010017,13233933,2020-09-01,1489.74
2,C0011810010020,99057968,2020-09-01,151.47
3,C0011810010021,80007276,2020-09-01,146.72
4,C0011810010024,13164076,2020-09-01,104.00
...,...,...,...,...
332725,S0081810310459,14092500,2020-09-30,3801.87
332726,S0081810310461,99065678,2020-09-30,5769.88
332727,S0081810310462,19029918,2020-09-30,736.88
332728,S0081810310463,13020033,2020-09-30,1475.20


# RFM analysis

In [7]:
# вытаскиваем последнюю дату, чтобы по ней сравнивать как давно была последняя покупка
last_date = rfm_analysis.InvoiceDate.max()

In [8]:
# проверим
last_date

Timestamp('2020-09-30 00:00:00')

In [9]:
# теперь посчитаем эту разницу в днях
rfm_analysis['recency'] = (last_date - rfm_analysis.InvoiceDate).dt.days

In [10]:
# находим данные для rfm
# группируем по пользователю, находим минимальную дату(последняя покупка), считаем кол-во покупок, считаем потраченную сумму
rfm_table = rfm_analysis \
            .groupby('CustomerCode') \
            .agg({'recency': 'min', 'InvoiceNo': 'count', 'Amount': 'sum'}) \
            .rename(columns={'InvoiceNo': 'frequency', 'Amount': 'monetary'})

In [11]:
# разделим пользователей на сегменты при помощи квантилей
# в переменную(quartile) запишем значение квантилей
quartile = rfm_table.quantile([0.25, 0.5, 0.75])
quartile = quartile.to_dict()
quartile

{'recency': {0.25: 2.0, 0.5: 8.0, 0.75: 16.0},
 'frequency': {0.25: 1.0, 0.5: 2.0, 0.75: 3.0},
 'monetary': {0.25: 765.0, 0.5: 1834.48, 0.75: 4008.84}}

In [12]:
# функция для разделения на сегменты при помощи квантилей
def for_r(x):
    if x <= quartile['recency'][0.25]:
        return 1
    elif x <= quartile['recency'][0.5]:
        return 2
    elif x <= quartile['recency'][0.75]:
        return 3
    else:
        return 4
    
def for_f(x):
    if x <= quartile['frequency'][0.25]:
        return 4
    elif x <= quartile['frequency'][0.5]:
        return 3
    elif x <= quartile['frequency'][0.75]:
        return 2
    else:
        return 1

def for_m(x):
    if x <= quartile['monetary'][0.25]:
        return 4
    elif x <= quartile['monetary'][0.5]:
        return 3
    elif x <= quartile['monetary'][0.75]:
        return 2
    else:
        return 1

In [13]:
# применяем функцию к столбцам
rfm_table['r'] = rfm_table.recency.apply(for_r)
rfm_table['f'] = rfm_table.frequency.apply(for_f)
rfm_table['m'] = rfm_table.monetary.apply(for_m)

In [14]:
# соединяем полученные разделения, тем самым получим сегменты
rfm_table['rfm_score'] = rfm_table['r'].map(str) + rfm_table['f'].map(str) + rfm_table['m'].map(str)

In [15]:
rfm_table.head(3)

,recency,frequency,monetary,r,f,m,rfm_score
CustomerCode,,,,,,,
02213019,19,1,1609.20,4,4,3,443
02213042,22,3,9685.48,4,2,1,421
02213071,29,1,415.00,4,4,4,444


In [16]:
print(f'{rfm_table.frequency.max()} максимальное кол-во покупок было совершено одним пользователем')

204 максимальное кол-во покупок было совершено одним пользователем


In [17]:
print(f"{rfm_table.rfm_score.value_counts().loc['311']} пользователей попало в сегмент 311")

1609 пользователей попало в сегмент 311


In [18]:
rfm_table.rfm_score.value_counts()

444    10624
111     9705
443     6729
344     6593
211     5847
       ...  
424       63
214       60
114       60
314       33
414        2
Name: rfm_score, Length: 64, dtype: int64